In [122]:
import os
import re
import glob
import json
import zipfile
import pandas as pd
import numpy as np
import pandas.io.formats.excel
from datetime import datetime, timedelta

In [123]:
def add_h_space(df_original, space=1):
    df_final = df_original.copy()
    width = df_final.shape[1]
    for _ in range(space):
        df_final.loc[len(df_final)] = [np.nan] * width
    return df_final


def add_v_space(df_original, space=1):
    df_final = df_original.copy()
    df_null = pd.DataFrame(columns=[np.nan] * space)
    df_final = pd.concat([df_final, df_null], axis=1)
    return df_final


def v_stack(df_list, space=3):
    max_width = max([x.shape[1] for x in df_list])

    df_first = df_list[0].copy().reset_index(drop=True)
    if df_first.shape[1] < max_width:
        df_first = add_v_space(df_first, max_width - df_first.shape[1])
    df_list_new = [df_first.copy()]
    
    df_null = pd.DataFrame(columns=df_first.columns)
    df_null = add_h_space(df_null, space)

    for df in df_list[1:]:
        df_temp = df.copy()
        if df_temp.shape[1] < max_width:
            df_temp = add_v_space(df, max_width - df_temp.shape[1])

        df_new = pd.DataFrame([df_temp.columns], columns=df_first.columns)
        df_temp.columns = df_first.columns

        df_list_new.append(df_null)
        df_list_new.append(df_new)
        df_list_new.append(df_temp.reset_index(drop=True))

    df_final = pd.concat(df_list_new).reset_index(drop=True)
    return df_final


def h_stack(df_list, space=3):
    df_first = df_list[0].copy().reset_index(drop=True)
    df_list_new = [df_first]
    df_null = pd.DataFrame({np.nan: [np.nan]})
    for df in df_list[1:]:
        df_temp = df.copy()
        for _ in range(space):
            df_list_new.append(df_null)
        df_list_new.append(df_temp.reset_index(drop=True))
    df_final = pd.concat(df_list_new, axis=1).reset_index(drop=True)
    return df_final

In [124]:
def timedelta_to_hh_mm(td):
    if pd.isna(td):
        return np.nan
    s = td.seconds
    return f'{str(int(s/3600)).zfill(2)}:{str(int((s%3600)/60)).zfill(2)}'

def timedelta_to_dd_hh_mm(td):
    if pd.isna(td):
        return np.nan
    d = td.days
    s = td.seconds
    return f'{str(d).zfill(2)}:{str(int(s/3600)).zfill(2)}:{str(int((s%3600)/60)).zfill(2)}'

def fix_time_slip(df):
    if df['GMT'] != 3 and df['Kaynak'] == 'K':
        df['T1'] = df['T1'] + timedelta(hours=3-df['GMT'])
    return df

In [125]:
pandas.io.formats.excel.ExcelFormatter.header_style= None
base_path = str(os.getcwd())

data_folder_path = base_path + '\\data\\'
data_file_name = '‎F🐨K Plak Takip ile WhatsApp Sohbeti.txt'
data_file_path = data_folder_path + data_file_name
json_file_path = data_folder_path + 'latest.json'

output_folder_path = base_path + '\\output\\'

zip_folder_path = os.path.expanduser('~') + '\\Downloads\\Phone Link\\'
zip_file_pattern = '*Plak Takip ile WhatsApp Sohbeti*.zip'

for c in [data_folder_path, output_folder_path, zip_folder_path]:
    os.makedirs(c, exist_ok=True)

In [126]:
search_pattern = zip_folder_path + zip_file_pattern
zip_file_list = glob.glob(search_pattern)
if zip_file_list:
    latest_zip_file_path = max(zip_file_list, key=os.path.getmtime)
    latest_zip_file_time = os.path.getmtime(latest_zip_file_path)

    try:
        with open(json_file_path, 'r') as f:
            last_info = json.load(f)
        last_update_time = last_info.get('mtime')
    except:
        last_update_time = None

    if last_update_time is None or latest_zip_file_time > last_update_time:
        with zipfile.ZipFile(latest_zip_file_path, 'r') as file:
            file.extractall(data_folder_path)

    latest_info = {
        'zip_path': latest_zip_file_path,
        'mtime': latest_zip_file_time
    }
    with open(json_file_path, 'w') as f:
        json.dump(latest_info, f, indent=4)

In [127]:
with open(data_file_path, 'r', -1, 'utf-8') as file:
    lines = file.readlines()

lines = [line.replace(' <Bu mesaj düzenlendi>', '') for line in lines]
lines = ['0' + line if line[1] == '.' else line for line in lines]

In [128]:
day_dict = {
    'Monday': 'Pazartesi',
    'Tuesday': 'Salı',
    'Wednesday': 'Çarşamba',
    'Thursday': 'Perşembe',
    'Friday': 'Cuma',
    'Saturday': 'Cumartesi',
    'Sunday': 'Pazar'
}

day_list = [x for x in day_dict.values()]

In [129]:
df_time_zones = pd.DataFrame(columns=['T1', 'GMT'])
df_data = pd.DataFrame(columns=['Tarih', 'Saat', 'Gün', 'İşlem', 'Kaynak', 'T1'])
df_skipped_data = pd.DataFrame(columns=['Tarih', 'Gün', 'Toplam Süre'])

for line in lines:
    timestamp = None
    if 'Berke Kaan Ülgen: GMT' in line:
        zone = int(line[-22:-20])
        timestamp = datetime.strptime(line[-18:-2], '%Y-%m-%d %H:%M')
        df_time_zones.loc[len(df_time_zones)] = [timestamp, zone]
        continue
    elif any(line.endswith(f'Filiz 🌱: {x}\n') for x in ['Çıkardım', 'Taktım', 'Değiştirdim']):
        source = 'F'
        timestamp = datetime.strptime(line[:16], '%d.%m.%Y %H:%M')
    elif any(f'Berke Kaan Ülgen: {x} (' in line for x in ['Çıkardım', 'Taktım', 'Değiştirdim']):
        source = 'K'
        timestamp = datetime.strptime(line[-18:-2], '%Y-%m-%d %H:%M')
    elif 'Berke Kaan Ülgen: Çıkarmadım (' in line:
        source = 'K'
        date = line[-12:-2]
        day = day_dict[datetime.strftime(datetime.strptime(line[-12:-2], '%Y-%m-%d'), '%A')]
        duration = timedelta()
        df_skipped_data.loc[len(df_skipped_data)] = [date, day, duration]
    elif 'Berke Kaan Ülgen: Takmıştım (' in line:
        timestamp = datetime.strptime(line[-18:-2], '%Y-%m-%d %H:%M')
        date = datetime.strftime(timestamp, '%Y-%m-%d')
        day = day_dict[datetime.strftime(timestamp, '%A')]
        duration = timedelta(hours=timestamp.hour, minutes=timestamp.minute)
        df_skipped_data.loc[len(df_skipped_data)] = [date, day, duration]
        timestamp = None
    if timestamp:
        date = datetime.strftime(timestamp, '%Y-%m-%d')
        time = datetime.strftime(timestamp, '%H:%M')
        day = day_dict[datetime.strftime(timestamp, '%A')]
        process = 'Çıkarıldı' if 'Çıkardım' in line else 'Takıldı' if 'Taktım' in line else 'Değiştirildi'
        df_data.loc[len(df_data)] = [date, time, day, process, source, timestamp]

df_time_zones = df_time_zones.sort_values('T1')
df_change = df_data.loc[df_data['İşlem'] == 'Değiştirildi'].copy().reset_index(drop=True)
df_data = df_data.loc[df_data['İşlem'] != 'Değiştirildi'].reset_index(drop=True)

df_data = df_data.sort_values('T1').reset_index(drop=True)
df_data = pd.merge_asof(df_data, df_time_zones, on='T1', direction='backward')
df_data = df_data.apply(fix_time_slip, axis=1)
df_data = df_data.drop('GMT', axis=1)

overflow_date_list = df_data.loc[(df_data['Tarih'] != df_data['Tarih'].shift()) & (df_data['İşlem'] == 'Takıldı'), 'Tarih'].values

for date in overflow_date_list:
    timestamp = datetime.strptime(date, '%Y-%m-%d')
    df_data.loc[len(df_data)] = [datetime.strftime(timestamp, '%Y-%m-%d'), '00:00', day_dict[datetime.strftime(timestamp, '%A')], 'Çıkarıldı', '',timestamp]
    timestamp -= timedelta(minutes=1)
    df_data.loc[len(df_data)] = [datetime.strftime(timestamp, '%Y-%m-%d'), '23:59', day_dict[datetime.strftime(timestamp, '%A')], 'Takıldı', '', timestamp]
df_data = df_data.sort_values(['T1', 'İşlem'], ascending=[True, False]).reset_index(drop=True)
df_data['Kaynak'] = df_data['Kaynak'].ffill()

df_data['T0'] = df_data['T1'].shift()
df_data.loc[df_data['İşlem'] == 'Takıldı', 'Geçen Süre'] = df_data['T1'] - df_data['T0']
df_data['Tarih Gün'] = df_data.apply(lambda x: x['Tarih'] + ' ' + x['Gün'], axis=1)

df_change = df_change.sort_values('T1').reset_index(drop=True)
df_change['T0'] = df_change['T1'].shift(-1)
df_change['Toplam Süre'] = df_change['T0'] - df_change['T1']
df_change['Tarih Gün'] = df_change.apply(lambda x: x['Tarih'] + ' ' + x['Gün'], axis=1)
df_change['Plak No'] = df_change.index + 1

df_data = pd.merge_asof(df_data, df_change[['T1', 'Plak No']], on='T1', direction='backward')

In [130]:
df_all = df_data[['Plak No', 'Tarih', 'Saat', 'Gün', 'İşlem', 'Kaynak', 'Geçen Süre']].copy()
df_all['Geçen Süre'] = df_all['Geçen Süre'].apply(timedelta_to_hh_mm)
df = df_all.copy()
df['Kontrol'] = (df['İşlem'] == df['İşlem'].shift()).replace(True, 'HATA').replace(False, '')
edl = df.loc[df['Kontrol'] == 'HATA', 'Tarih'].unique()
df.loc[df['Tarih'].isin()].replace(np.nan, '').tail(20) if len(edl) else df.replace(np.nan, '').tail(10)

,Plak No,Tarih,Saat,Gün,İşlem,Kaynak,Geçen Süre,Kontrol
464,13,2025-08-22,12:45,Cuma,Çıkarıldı,F,,
465,13,2025-08-22,13:08,Cuma,Takıldı,F,00:23,
466,13,2025-08-22,20:39,Cuma,Çıkarıldı,F,,
467,13,2025-08-22,22:04,Cuma,Takıldı,F,01:25,
468,13,2025-08-23,13:37,Cumartesi,Çıkarıldı,F,,
469,13,2025-08-23,14:13,Cumartesi,Takıldı,F,00:36,
470,13,2025-08-23,19:02,Cumartesi,Çıkarıldı,F,,
471,13,2025-08-23,20:00,Cumartesi,Takıldı,K,00:58,
472,13,2025-08-24,11:34,Pazar,Çıkarıldı,F,,
473,13,2025-08-24,12:30,Pazar,Takıldı,K,00:56,


In [131]:
df_sum = df_data[['Tarih', 'Gün', 'Geçen Süre']].groupby(['Tarih', 'Gün']).sum()
df_sum = df_sum.reset_index()
df_sum.columns = ['Tarih', 'Gün', 'Toplam Süre']
df_sum = pd.concat([df_sum, df_skipped_data], ignore_index=True)
df_sum['T1'] = df_sum['Tarih'].apply(lambda x: datetime.strptime(x + ' 23:59', '%Y-%m-%d %H:%M'))
df_sum = df_sum.sort_values('T1').reset_index(drop=True)
df_sum = pd.merge_asof(df_sum, df_change[['T1', 'Plak No']], on='T1', direction='backward')

df_sum_sum = df_sum[['Plak No', 'Toplam Süre']].groupby('Plak No').sum().reset_index()
df_sum_sum.columns = ['Plak No', 'Çıkarılan Süre']

df_sum['Toplam Süre'] = df_sum['Toplam Süre'].apply(timedelta_to_hh_mm)
df_sum['Gün No'] = df_sum.index + 1
df_sum = df_sum[['Plak No', 'Gün No', 'Tarih', 'Gün', 'Toplam Süre']]
df_sum.tail(10)

,Plak No,Gün No,Tarih,Gün,Toplam Süre
118,12,119,2025-08-15,Cuma,01:53
119,12,120,2025-08-16,Cumartesi,02:15
120,12,121,2025-08-17,Pazar,02:27
121,13,122,2025-08-18,Pazartesi,01:20
122,13,123,2025-08-19,Salı,02:01
123,13,124,2025-08-20,Çarşamba,01:45
124,13,125,2025-08-21,Perşembe,01:37
125,13,126,2025-08-22,Cuma,02:49
126,13,127,2025-08-23,Cumartesi,01:34
127,13,128,2025-08-24,Pazar,00:56


In [132]:
df_cycle = df_change[['Plak No', 'Tarih', 'Saat', 'Gün', 'İşlem', 'Toplam Süre']].copy()
df_cycle = pd.merge(df_cycle, df_sum_sum, on='Plak No', how='left')
df_cycle['Çıkarılan Süre'] = df_cycle['Çıkarılan Süre'].apply(timedelta_to_hh_mm)
df_cycle['Toplam Süre'] = df_cycle['Toplam Süre'].apply(timedelta_to_dd_hh_mm)
df_cycle = df_cycle[['Plak No', 'Tarih', 'Saat', 'Gün', 'İşlem', 'Çıkarılan Süre', 'Toplam Süre']]
df_cycle.replace(np.nan, '')

,Plak No,Tarih,Saat,Gün,İşlem,Çıkarılan Süre,Toplam Süre
0,1,2025-04-19,11:25,Cumartesi,Değiştirildi,14:15,10:23:41
1,2,2025-04-30,11:06,Çarşamba,Değiştirildi,14:05,10:00:25
2,3,2025-05-10,11:31,Cumartesi,Değiştirildi,13:18,10:01:16
3,4,2025-05-20,12:47,Salı,Değiştirildi,17:35,10:00:42
4,5,2025-05-30,13:29,Cuma,Değiştirildi,11:40,10:00:57
5,6,2025-06-09,14:26,Pazartesi,Değiştirildi,13:51,09:20:43
6,7,2025-06-19,11:09,Perşembe,Değiştirildi,15:11,10:05:27
7,8,2025-06-29,16:36,Pazar,Değiştirildi,14:10,09:16:17
8,9,2025-07-09,08:53,Çarşamba,Değiştirildi,11:52,10:03:07
9,10,2025-07-19,12:00,Cumartesi,Değiştirildi,10:44,09:23:01


In [133]:
output_file_name = datetime.strftime(datetime.now(), '%Y-%m-%d %M-%H Plak Özet')
output_file_path = output_folder_path + output_file_name + '.xlsx'
writer = pd.ExcelWriter(output_file_path, engine = 'xlsxwriter')
df_out = h_stack([df_all, df_sum, df_cycle], 2)
df_out.to_excel(writer, sheet_name='Plak', index=False)
writer.sheets['Plak'].hide_gridlines(option=2)
writer.close()
os.startfile(output_file_path)